## Setup
Install accompanying pinon library. It is better to install in dev mode in the conda environment with `pip install -e .` Jupyter server must restart to see changes to conda environment.
Auto reload the pinon module.  Other imports may be ignored for auto reload e.g.  `%aimport -simfin`. Note use of option 2 for auto reload and excluding other modules, rather than option 1 and including only reloadable modules.  This is because option 1 does not work correctly with aliased imports.

In [1]:
# !pip install -e ..

%load_ext autoreload
%autoreload 2
%aimport -pandas
%aimport -simfin
%aimport -os

%aimport pinon

%matplotlib inline

# imports
import pandas as pd
import os

import pinon as pn
import simfin as sf
import simfin.names as sf_cols  # column name shortcuts

config = pn.Config('master')

Config file found at: /Users/daved/Dev/pinon/config/master.xlsx
Simfin data directory: /Users/daved/Dev/pinon/simfin_data


In [29]:
ts = config.parse_target_sheet('V', 10)
ts

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3625, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'AXP'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/9p/f2tvcxd55k125ycs4hyyx6800000gn/T/ipykernel_96

In [8]:
c = pn.Companies()

In [30]:
res = c.get_company('AXP')
res

SimFinId                     413080
Company Name    AMERICAN EXPRESS CO
IndustryId                 104007.0
Name: AXP, dtype: object

In [31]:
f = pn.Fundamentals()
f.get_last_report_date('AXP')

KeyError: 'AXP'